**click link below to install ChromeDriver bef running other code**


[Install ChromeDriver](https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4)

[How to Scrape One Page from Edmunds](https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab)

In [1]:
#install selenium
!pip install selenium

In [2]:
#print current directory for Jupyter NB
import os
import sys
os.path.dirname(sys.executable)

'/Users/hau/Anaconda3/anaconda3/bin'

In [3]:
import pandas as pd
from selenium import webdriver

In [4]:
#set up ChromeDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options = chrome_options)

In [16]:
#example: scrape one user id
driver.get('https://forums.edmunds.com/discussion/50806/general/x/car-subscription-vs-lease-vs-purchase')
userid_element = driver.find_elements_by_xpath('//*[@id="Comment_5488847"]/div/div[2]/div[1]/span[1]/a[2]')[0]
userid = userid_element.text

print(userid)

PF_Flyer


In [6]:
#scraper for a single web page

def page_scraper(url):
    driver.get(url)
    #output df
    comments = pd.DataFrame(columns = ['Date','user_id','comments']) 

    #comment id list
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    comment_ids = []
    
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    #extract date/id/message from each comment unit
    for x in comment_ids:
        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')
        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text
        #Extract Message for each user on a page
        user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]/div/div[1]')[0]
        comment = user_message.text
        #put all three in one row of the output df
        comments.loc[len(comments)] = [date,userid,comment]
        
    return comments


#test
page_1 = page_scraper('https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0/p2')
page_1.head(3)

,Date,user_id,comments
0,"April 15, 2007 8:36PM",elroy5,"No, I have not driven a Sonata. I have driven ..."
1,"April 15, 2007 11:40PM",plekto,Of course the Buick LaCrosse qualifies. It cer...
2,"April 16, 2007 12:17AM",punkr77,One thing that drops the Maxima off my shoppin...


In [7]:
#generate a list of urls for all pages in one thread

def page_urls_generater(thread_url, page_count):
    page_urls = [thread_url, ]
    for i in range(2, page_count + 1):
        page_urls.append(thread_url + '/p' + str(i))
    
    return page_urls

#test
test_thread = 'https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0' 
test_page_count = 543
test_url_list = page_urls_generater(test_thread, test_page_count)
len(test_url_list)

543

In [8]:
#scraper for one thread - The Scraper

def scraper(thread_url, page_count):
    #output df
    comments = pd.DataFrame(columns = ['Date','user_id','comments']) 
    
    #iterate over page urls
    for url in page_urls_generater(thread_url, page_count):
        page_comm = page_scraper(url)
        comments = comments.append(page_comm, ignore_index = True)
        
    return comments


In [9]:
#test
test_thread = 'https://forums.edmunds.com/discussion/5779/dodge/ram-pickup-1500/dodge-ram-transmission-problems' 
test_page_count = 42
test_comments = scraper(test_thread, test_page_count)
test_comments

,Date,user_id,comments
0,"September 1, 2006 12:33PM",newdodgeowner,I have a 2001 Dodge Ram with a 5.9. I noticed ...
1,"September 2, 2006 1:22AM",rollerdoo,I was told it is the lock up converter. I also...
2,"September 9, 2006 11:36AM",carnut17,Can anyone please help !!! I purcased a 98 ram...
3,"September 9, 2006 6:02PM",KCRam,"When my 96 3500 Cummins would start slipping, ..."
4,"September 10, 2006 10:23AM",carnut17,"Thanks so much kcram, for the advice. I will h..."
...,...,...,...
2057,"November 6, 2018 6:49AM",jamjon,jamjon said:\nshow previous quotes\nI had my m...
2058,"January 3, 2019 8:06PM",RamChick34,Hey guys. I have an 01 1500 with the 5.9 and 4...
2059,"January 3, 2019 8:16PM",RamChick34,"gaidinbdj said:\nAlright, I'll start off by sa..."
2060,"July 1, 2019 11:35PM",Pyrosrule69,I know this is probably a dead thread. But I h...


In [11]:
#test_comments.to_csv('test_dodge_thread.csv', index = False)